In [ ]:
''' Description: 
Performing stochastic gradient descent to find a global function on P1 that satisfies given differentiability conditions (analogous to a finding metric which satisfies Ricci-flatness in tutorial 3)...
P1 is built from C2(z0,z1), and hence is parameterised by these homogeneous coordinates.
The differentiability conditions our desired function, F (modelled by f), satisifes are:
    1) \frac{\partial F}{\partial z_0}             = \frac{\overline{z_1}(|z_0|^2+|z_1|^2-\overline{z_0})}{(|z_0|^2+|z_1|^2)^2}
    2) \frac{\partial F}{\partial \overline{z_0}}  = \frac{-z_0^2*\overline{z_1}}{(|z_0|^2+|z_1|^2)^2}
    3) \frac{\partial F}{\partial z_1}             = \frac{-z_0*\overline{z_1}^2}{(|z_0|^2+|z_1|^2)^2}
    4) \frac{\partial F}{\partial \overline{z_1}}  = \frac{z_0(|z_0|^2+|z_1|^2-z_1)}{(|z_0|^2+|z_1|^2)^2}
'''

In [ ]:
#Import libraries
import numpy as np
import matplotlib.pyplot as plt

#Set the random seed (None for no seeding)
seed = 3


In [ ]:
#Point sampling on P1
number_of_points = int(1e3)

#Homogeneous points
if seed: np.random.seed(seed)
sphere_points = np.random.normal(loc=0.0, scale=1.0, size=(number_of_points,4))
sphere_points /= np.linalg.norm(sphere_points,axis=1)[:, np.newaxis]
homogeneous_points = np.array([[pt_set[0]+1j*pt_set[1], pt_set[2]+1j*pt_set[3]] for pt_set in sphere_points])
del(sphere_points)

#Split into train, validation, test
split_proportions = [0.7,0.1,0.2] #..note only uses train and validation proportions, assumes test as the remainder
train_points      = homogeneous_points[:int(np.floor(number_of_points*split_proportions[0]))]
validation_points = homogeneous_points[int(np.floor(number_of_points*split_proportions[0])):int(np.floor(number_of_points*(split_proportions[0]+split_proportions[1])))]
test_points       = homogeneous_points[int(np.floor(number_of_points*(split_proportions[0]+split_proportions[1]))):]


In [ ]:
#Define the ansatz for the function in homogeneous coordinates
max_order = 1 #...select the order to extend the ansatz too
z_0, zb_0, z_1, zb_1 = var('z_0, zb_0, z_1, zb_1', domain='complex') #...define the homogeneous variables (note define the conjugates as separate variables so can perform partial differentiation)

#Initialise the function
f(z_0,zb_0,z_1,zb_1) = 0
#Loop through the degree orders (note following cells currently only work for max_order=1)
for i in range(1,max_order+1):
    #Define the coefficients for this order as variables
    c = matrix(SR, i+1, i+1, lambda alpha,beta: var('c_{}_{}_{}'.format(i,alpha,beta), domain='real'))
    #print('Coefficients: {c}')
    
    #Initialise the function contribution for this order
    fi(z_0,z_1) = 0
    #Loop through all homogeneous polynomials in both holomorphic & antiholomorphic parts, and add them with their respective general coefficient
    for j1 in range(i+1):
        for j2 in range(i+1):
            fi += c[j1,j2] * z_0**j1 * z_1**(i-j1) * zb_0**j2 * zb_1**(i-j2) / (z_0*zb_0 + z_1*zb_1)**i
    
    #Add this order's ansatz to the full ansatz
    f += fi 
    
#Print the final ansatz form
print(f.full_simplify())

In [ ]:
#Define the differentiability conditions
dFdz0(z_0,z_1)  =  (z_1*conjugate(z_1)**2)/(z_0*conjugate(z_0)+z_1*conjugate(z_1))**2
dFdzb0(z_0,z_1) = (-z_0**2*conjugate(z_1))/(z_0*conjugate(z_0)+z_1*conjugate(z_1))**2
dFdz1(z_0,z_1)  = (-z_0*conjugate(z_1)**2)/(z_0*conjugate(z_0)+z_1*conjugate(z_1))**2
dFdzb1(z_0,z_1) =  (z_0**2*conjugate(z_0))/(z_0*conjugate(z_0)+z_1*conjugate(z_1))**2

#Compute symbolic loss function
LossA = norm(f.diff(z_0 ).substitute(zb_0=conjugate(z_0),zb_1=conjugate(z_1)) - dFdz0)
LossB = norm(f.diff(zb_0).substitute(zb_0=conjugate(z_0),zb_1=conjugate(z_1)) - dFdzb0)
LossC = norm(f.diff(z_1 ).substitute(zb_0=conjugate(z_0),zb_1=conjugate(z_1)) - dFdz1)
LossD = norm(f.diff(zb_1).substitute(zb_0=conjugate(z_0),zb_1=conjugate(z_1)) - dFdzb1)
Total_Loss = LossA + LossB + LossC + LossD
Total_Loss = Total_Loss.full_simplify()
#print(Total_Loss)

#Define the loss function (evaluating for the given coefficients at the considered point)
def Loss(constants, point):
    return float(real_part(Total_Loss.substitute(c_1_0_0=constants[0], c_1_0_1=constants[1], c_1_1_0=constants[2], c_1_1_1=constants[3]).substitute(z_0=point[0],z_1=point[1])))
    
#Compute symbolic loss gradient wrt coefficients
gradient_fn  = [Total_Loss.diff(c_1_0_0).full_simplify(),Total_Loss.diff(c_1_0_1).full_simplify(),Total_Loss.diff(c_1_1_0).full_simplify(),Total_Loss.diff(c_1_1_1).full_simplify()]

#Define the loss gradient function (evaluating for the given coefficients at the considered point)
def LossGradient(constants,point):
    gradient_vec = [coeff_grad.substitute(c_1_0_0=constants[0], c_1_0_1=constants[1], c_1_1_0=constants[2], c_1_1_1=constants[3]) for coeff_grad in gradient_fn]
    gradient     = np.real(np.array([coeff_grad.substitute(z_0=point[0], z_1=point[1]) for coeff_grad in gradient_vec],dtype=complex))
    return gradient

#Test
example_coeffs, example_point = [-1,2,3,4], [3*I,5+I]
print(f'Example coefficients {example_coeffs}, and point {example_point}')
print(f'Loss: {Loss(example_coeffs,example_point)}')
print(f'Loss gradient: {LossGradient(example_coeffs,example_point)}')

In [ ]:
#Initialise coefficients
if seed: np.random.seed(seed)
coefficients = np.random.normal(loc=0.0, scale=3.0, size=4)
print(f'Coefficients: {coefficients}\n')

#Define SGD hyperparameters
batch_size = 32
number_of_epochs = 10
learning_rate = 1e-2
momentum = 0.8
tolerance = 1e-8

#Perform the SGD
epoch_losses = []
for epoch in range(number_of_epochs):
    #Shuffle the data (so batches change each epoch)
    np.random.shuffle(train_points)
    np.random.shuffle(validation_points)
    
    #Loop over batches
    print(f'Epoch: {epoch+1}\nProgress: ',end='')
    batches_losses = []
    coeff_step = 0
    for batch_count, batch_idx in enumerate(range(np.floor(len(train_points)/batch_size).astype(int))):
        #Compute loss for the batch
        batches_losses.append(np.mean([Loss(coefficients,train_points[idx]) for idx in range(batch_idx*batch_size,min(number_of_points+1,(batch_idx+1)*batch_size))]))
        #print(f'\nBatch {batch_idx+1} Loss:\t{batches_losses[-1]}')
        
        #Compute the gradient across the batch, and the respective step in the coefficients
        grad = np.mean([LossGradient(coefficients,train_points[idx]) for idx in range(batch_idx*batch_size,min(number_of_points+1,(batch_idx+1)*batch_size))],axis=0)
        coeff_step = - learning_rate*grad
        #coeff_step = - ((number_of_epochs-epoch)/number_of_epochs)*learning_rate*grad #...with decay
        #coeff_step = momentum*coeff_step - learning_rate*grad #...with momentum
        #coeff_step = momentum*coeff_step - ((number_of_epochs-epoch)/number_of_epochs)*learning_rate*grad #...with momentum & decay
        
        #Early-stop the descent if the step size too small
        if np.all(np.abs(coeff_step) <= tolerance):
            print('...early stopping',end=' ')
            break

        #Update the coefficients
        coefficients += coeff_step
        if batch_count%int(len(train_points)/(20*batch_size)) == 0: print('=',end='')
    print('x')
    
    #Compute epoch losses
    train_loss = np.mean(batches_losses) 
    validation_loss = np.mean([Loss(coefficients,val_point) for val_point in validation_points])
    print(f'Training Loss:   {train_loss}\nValidation Loss: {validation_loss}\nCoefficients: {coefficients}\n')
    epoch_losses.append([train_loss,validation_loss])
    
#Run testing
epoch_losses = np.array(epoch_losses)
test_loss = np.mean([Loss(coefficients,test_point) for test_point in test_points])
print(f'##########\nFinal Test Loss: {test_loss}\nFinal Coefficients: {np.round(coefficients)}')

In [ ]:
#Loss Plotting
plt.figure()
if number_of_epochs == 1: 
    plt.plot(range(1,len(batches_losses)+1),batches_losses,label='Batch')
else:
    plt.plot(range(1,len(epoch_losses)+1),epoch_losses[:,0],label='Train')
    plt.plot(range(1,len(epoch_losses)+1),epoch_losses[:,1],label='Validation')
plt.title('Losses')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim(0)
plt.xlim(0)
plt.legend(loc='best')
plt.grid()
plt.tight_layout()
#plt.savefig('Losses.pdf')

In [ ]:
#Potential Extensions: 
# ~ Test how performance changes with different point sampling, and number of points.
# ~ Extend SGD (trial other learning rates, adapt decay rate (exponential) & momentum (accumulated), generalise to RMSProp & Adam).
# ~ Extend the loss functions to higher order Ln losses (or lower with norm --> abs), add regularisation, trial other loss styles.
# ~ Trial losses based on 2nd order derivatives (closer to Ricci scalar condition): compute by hand, then hardcode in.
# ~ Trial other P1 functions based on the same ansatz and learn those: set the desired function's coefficients, compute derivatives by hand, hardcode in.
# ~ Trial sage's inbuilt 'desolve' function: https://doc.sagemath.org/html/en/tutorial/tour_algebra.html#solving-differential-equations
# ~ Generalise code usability by converting code cells to functions.